In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from scipy.integrate import quad
from scipy.special import hermite, factorial

In [ ]:
import matplotlib
matplotlib.rcParams['animation.embed_limit']=10*matplotlib.rcParams['animation.embed_limit']

In [ ]:
sp.init_printing(use_latex='mathjax')
x=sp.symbols('x',real=True)
omega=sp.symbols('\\omega', real=True)
m=sp.symbols('m',positive=True)
#a=sp.symbols('a',positive=True) to a jest niepotrzebne wsm
k=sp.symbols('k',positive=True)
hbar=sp.symbols('hbar',positive=True)
n=sp.symbols('n',integer=True, positive=True)
E=sp.symbols('E',real=True)
t=sp.symbols('t',real=True)
En=hbar*omega*(n + 0.5)
Psi=(1/(sp.sqrt(2**n * sp.factorial(n)))) *(m* omega/(sp.pi*hbar))**(1/4)* sp.exp(-m*omega*x**2/(2*hbar)) * sp.hermite(n,sp.sqrt(m*omega/hbar)*x)

Psi=Psi*sp.exp(-sp.I*En*t/hbar)

display(Psi)

In [ ]:
Psi_func = sp.lambdify((x, t, n),
                       Psi.subs({hbar: 1, m: 1, omega: 1}),
                       modules=['numpy', 'scipy'])


xs=np.linspace(-4,4,1000)
plt.plot(xs,Psi_func(xs,0,1).real, label='n=1')
plt.plot(xs,Psi_func(xs,0,2).real, label='n=2')
plt.plot(xs,Psi_func(xs,0,3).real, label='n=3')

plt.title('Stany stacjonarne kwantowego oscylatora harmonicznego ($t=0$)')
plt.legend()
plt.xlabel('Położenie (x)')
plt.ylabel('Re[$\Psi(x,t)$]')
plt.axhline(0, color='black', lw=1, alpha=0.5)
plt.grid(alpha=0.4, linestyle='--', color='blue')
plt.show()




In [ ]:
# WIZUALIZACJA STANÓW STACJONARNYCH
n = 6 # MOZNA ZNIENIAC.
E_n=n + 0.5
xs=np.linspace(-5,5,1000)
V=0.5*xs**2
plt.style.use('dark_background')
fig=plt.figure()

plt1,=plt.plot([],[], color="royalblue",label='Re[$\Psi$] + $E_n$') #odpakowanie listy
plt2,=plt.plot([],[], color= "tomato",label='Im[$\Psi$] + $E_n$') #puste wykresiki
#plt3=plt.fill_between([],[],[], zorder=2, alpha=0.5, label='$|\Psi|^2$ + $E_n$') #gestosc prawdo

plt.plot(xs,V,color="yellow",linestyle="--", alpha=0.5,label="potencjał $V(x)$")
plt.axhline(E_n, color='white', linestyle=':',  label='E_n')

plt.xlabel('x')
plt.ylabel('$\\Psi(x,t)$')
plt.grid(alpha=0.4, linestyle='--')
plt.legend(loc="lower right")

#trzeba przeskalowac puste wykresiki
plt.xlim(-5,5)
plt.ylim(0,E_n + 2)
plt.close()
dt=0.005

def frame(i):
  t=i*dt
  ys=Psi_func(xs,t,n)
  plt1.set_data(xs,ys.real + E_n)
  plt2.set_data(xs,ys.imag + E_n)
  #plt3.set_data(xs,np.abs(ys)**2 + E_n,0)
  return plt1 , plt2,# plt3,

anim = FuncAnimation(fig,frame,frames=800,interval=20,blit=True)
HTML(anim.to_jshtml() )

In [ ]:
# PACZKA FALOWA

def lincomb(xs,t,coeffs):
  norm=np.sqrt(sum(np.abs(c)**2 for c in coeffs.values()))
  psi_sum=np.zeros_like(xs,dtype=complex)
  for n, c in coeffs.items():
    psi_sum=psi_sum + c*Psi_func(xs,t,n)
  return psi_sum/(norm)


coeffs = {ebe: 1 for ebe in range(6)} #tu ten slownik a nizej sr arytm
E_avg = sum(n + 0.5 for n in coeffs.keys()) / len(coeffs)

xs = np.linspace(-6, 6, 1000)
V = 0.5 * xs**2

fig, ax = plt.subplots(figsize=(10, 6))
plt.style.use('dark_background')


ax.plot(xs, V, color="yellow", linestyle="--", alpha=0.3)
ax.axhline(E_avg, color='white', linestyle=':', alpha=0.2)


#plt1, = ax.plot([], [], color="cyan", lw=2, label='$|\Psi|^2 + E_{avg}$')
plt1=plt.fill_between([],[],[],zorder=2, alpha=0.7,color="red")

ax.set_xlim(-6, 6)
ax.set_ylim(0, E_avg + 3)
ax.set_xlabel('x')
ax.set_ylabel('Energia / Gęstość')
ax.legend()
ax.grid(alpha=0.3)
plt.close()
dt=0.05
def frame(i):
    t = i*dt

    ys =np.abs(lincomb(xs, t, coeffs))**2
    plt1.set_data(xs,E_avg, ys + E_avg)
    return plt1,


anim = FuncAnimation(fig, frame, frames=200, interval=30, blit=True)
HTML(anim.to_jshtml())

In [ ]:
#wizualizacja gęstości prawdopodobieństwa 2 stanów energetycznych w dwuwymiarowym oscylatorze harmonicznym

hbar = 1.0
m = 1.0
#można częstowliwość zmienić, to jest ważne- ma wpływ na szerokość oscylatora
omega_x = 1
omega_y = 1.2

#siatka punktów
N = 200
x = np.linspace(-5, 5, N)
y = np.linspace(-5, 5, N)
X, Y = np.meshgrid(x, y)


def psi_1D(n, x, omega):
  xi = np.sqrt(m * omega / hbar) * x
  Hn = hermite(n)(xi)
  norm = (m*omega/ (np.pi* hbar))**0.25/np.sqrt(2**n * factorial(n))
  return norm * Hn * np.exp(-xi**2 / 2)


def psi_2D(nx, ny):
  return psi_1D(nx, X, omega_x) * psi_1D(ny, Y, omega_y)


# STANY, można zmienić

#Stan 1
nx1= 1
ny1 =1
#stan 2
nx2 =2
ny2 =0

psi_1 = psi_2D(nx1, ny1)
psi_2 = psi_2D(nx2, ny2)

E_1 = hbar * ((nx1*2+1)*omega_x/2 + (ny1*2+1)*omega_y/2)
E_2 = hbar * ((nx2*2+1)*omega_x/2 + (ny2*2+1)*omega_y/2)


fig, ax = plt.subplots()
img = ax.imshow(np.abs(psi_1)**2,extent=[x.min(), x.max(), y.min(), y.max()],origin="lower",cmap="inferno")
ax.set_title("Wizualizacja gęstości prawdopodobieństwa 2 stanów energetycznych w dwuwymiarowym oscylatorze harmonicznym")
ax.set_xlabel("x")
ax.set_ylabel("y")
fig.colorbar(img, ax=ax)

def update(frame):
  t = frame * 0.05
  Psi = (psi_1 * np.exp(-1j *E_1* t/hbar) +psi_2 * np.exp(-1j* E_2 * t/ hbar)) / np.sqrt(2)
  rho =np.abs(Psi)**2
  img.set_array(rho)
  return [img]

ani = FuncAnimation(fig,update,frames=300,interval=40,blit=True)

display(HTML(ani.to_jshtml()))